# 智能体框架 Framework

照例先感谢Datawhale的课程：https://datawhalechina.github.io/hello-agents/#/./chapter6/%E7%AC%AC%E5%85%AD%E7%AB%A0%20%E6%A1%86%E6%9E%B6%E5%BC%80%E5%8F%91%E5%AE%9E%E8%B7%B5

课程介绍了四种主流的智能体框架：

- Autogen

- AgentScope

- CAMEL

- LangGraph

## LangGraph



In [3]:
!pip install langgraph openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/90.6 kB ? eta -:--:--  Downloading pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 210.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 210.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/57.7 kB ? eta -:--:--  Downloading tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 177.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 177.7 kB/s eta 0:00:00a 0:00:01
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 4.8 MB/s eta 0:00:00     ━━━━━━━━━

In [7]:
from typing import TypedDict
import os

from openai import OpenAI
from langgraph.graph import StateGraph, END
from IPython.display import Markdown, display

# === Environment and OpenAI client ===
# Use environment variables: OPENAI_API_KEY (required), OPENAI_API_BASE (optional), OPENAI_MODEL (optional)
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not set in environment. Please set it before running this notebook.")

# default base URL as requested
base_url = os.getenv("OPENAI_API_BASE", "https://xiaoai.plus/v1")
model = os.getenv("OPENAI_MODEL", "gpt-5-mini")

# Initialize client
client = OpenAI(api_key=api_key, base_url=base_url)

# A simple process log to collect intermediate information for learning/inspection
process_log = []

# Small wrapper to call the chat completions endpoint and return text
def call_llm(prompt: str, show_prompt: bool = True) -> str:
    """Call the configured LLM and return the assistant text.
    Shows prompt and response as notebook-friendly Markdown for learning purposes.
    """
    if show_prompt:
        display(Markdown("**LLM prompt:**\n```text\n" + prompt + "\n```"))

    resp = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )

    # Note: response format uses choices[0].message.content
    text = resp.choices[0].message.content

    # show the response
    display(Markdown("**LLM response:**\n" + ("```text\n" + text + "\n```" if text else "(empty)")))

    # record into process log
    process_log.append({"prompt": prompt, "response": text})

    return text


# 1️⃣ 定义 State
class ArticleState(TypedDict, total=False):
    topic: str
    outline: str
    draft: str
    refined_draft: str


# 2️⃣ 定义 Nodes（每个只做一件事）
def make_outline(state: ArticleState) -> ArticleState:
    """Generate a brief outline for a given topic using the LLM."""
    topic = state.get("topic", "")
    prompt = (
        f"You are an assistant that writes short article outlines.\n"
        f"Produce a concise outline (3-5 bullet points) for an article about: {topic}\n"
        "Return the outline as plain text, each bullet on a new line."
    )

    display(Markdown(f"### Node: make_outline — topic: **{topic}**"))
    outline = call_llm(prompt)

    # store intermediate state for visibility
    display(Markdown("**Outline (stored to state):**\n\n" + ("```text\n" + outline + "\n```" if outline else "(empty)")))

    return {"outline": outline}


def write_draft(state: ArticleState) -> ArticleState:
    """Generate a draft paragraph based on the outline using the LLM."""
    outline = state.get("outline", "")
    display(Markdown("### Node: write_draft — using outline above"))

    prompt = (
        "You are an assistant that writes article paragraphs.\n"
        f"Write a clear, readable article paragraph (around 150-300 words) based on the following outline:\n{outline}\n"
        "Keep language neutral and suitable for a general audience."
    )
    draft = call_llm(prompt)

    display(Markdown("**Draft (stored to state):**\n\n" + ("```text\n" + draft + "\n```" if draft else "(empty)")))

    return {"draft": draft}


def refine_draft(state: ArticleState) -> ArticleState:
    """Refine the existing draft into a 'New Scientist' style: concise, engaging, lightly journalistic but scientifically grounded."""
    draft = state.get("draft", "")
    outline = state.get("outline", "")
    display(Markdown("### Node: refine_draft — refining draft into ‘New Scientist’ style"))

    prompt = (
        "You are an editor for a popular science magazine (New Scientist style).\n"
        "Rewrite the provided draft to be concise, engaging, and accessible to a general audience.\n"
        "Keep scientific accuracy, use clear explanations and a slightly journalistic tone, and avoid jargon where possible.\n"
        "Limit the refined draft to about 150-250 words.\n\n"
        f"Outline:\n{outline}\n\nDraft:\n{draft}\n\n"
        "Return only the refined article text (no extra commentary)."
    )

    refined = call_llm(prompt)

    display(Markdown("**Refined draft (stored to state):**\n\n" + ("```text\n" + refined + "\n```" if refined else "(empty)")))

    return {"refined_draft": refined}


# 3️⃣ 构建 Graph
def build_graph():
    graph = StateGraph(ArticleState)

    graph.add_node("outline", make_outline)
    graph.add_node("draft", write_draft)
    graph.add_node("refine", refine_draft)

    graph.set_entry_point("outline")
    graph.add_edge("outline", "draft")
    graph.add_edge("draft", "refine")
    graph.add_edge("refine", END)

    return graph.compile()


# 4️⃣ 运行（示例）
if __name__ == "__main__":
    app = build_graph()

    initial_state = {"topic": "elderly smartphone usage and health"}

    display(Markdown("## Running the agent graph"))
    display(Markdown("**Initial state:**\n\n```json\n" + str(initial_state) + "\n```"))

    final_state = app.invoke(initial_state)

    # Present final state as Markdown for better readability
    display(Markdown("## Final state (summary)"))
    topic = final_state.get("topic", initial_state.get("topic", "(none)"))
    outline = final_state.get("outline", "(none)")
    draft = final_state.get("draft", "(none)")
    refined = final_state.get("refined_draft", "(none)")

    display(Markdown(f"### Topic:\n\n**{topic}**"))
    display(Markdown("### Outline:\n\n" + ("```text\n" + outline + "\n```" if outline else "(none)")))
    display(Markdown("### Draft:\n\n" + ("```text\n" + draft + "\n```" if draft else "(none)")))
    display(Markdown("### Refined Draft (New Scientist style):\n\n" + ("```text\n" + refined + "\n```" if refined else "(none)")))

    # Optionally show the process log as an ordered list
    display(Markdown("## Process log (LLM calls in order)"))
    for i, entry in enumerate(process_log, 1):
        resp_len = len(entry.get('response') or "")
        display(Markdown(f"**Step {i}** — stored response length: {resp_len} chars"))

    display(Markdown("---\nRun complete."))


## Running the agent graph

**Initial state:**

```json
{'topic': 'elderly smartphone usage and health'}
```

### Node: make_outline — topic: **elderly smartphone usage and health**

**LLM prompt:**
```text
You are an assistant that writes short article outlines.
Produce a concise outline (3-5 bullet points) for an article about: elderly smartphone usage and health
Return the outline as plain text, each bullet on a new line.
```

**LLM response:**
```text
- Benefits: how smartphones support seniors through social connection, telehealth, medication reminders, and emergency alerts.

- Health risks and mitigations: address eye strain, sleep disruption, and posture issues with accessibility settings, blue-light filters, larger text, and regular breaks.

- Practical guidelines for adoption: recommend senior-friendly devices and apps, voice assistants, clear onboarding, and ongoing tech support with caregiver involvement.

- Safety, privacy, and support: emphasize data privacy, scam awareness, built-in safety features (fall detection, SOS), and access to local training resources.
```

...

## Process log (LLM calls in order)

**Step 1** — stored response length: 630 chars

**Step 2** — stored response length: 1551 chars

**Step 3** — stored response length: 1442 chars

---
Run complete.